In [127]:
import random
import re
import string
from collections import Counter
import math

class Wordle(object):

    def __init__(self):
        self._answers = self.readfiles('2315_possible_answers.txt')
        self._allowed = self.readfiles('12972_allowed_words.txt')
        self._allowedWordEntropy = {line.split()[0] : float(line.split()[1])  for line in self.readfiles('12972_allowed_words_entroy.txt')}
        self._passed = self._allowed
        self.updateEnropy()

    # ---    Word Guessed  -----------   
        #self._wordleWord = random.choice(self._answers)
        #print(self._wordleWord.upper())

    #----   Reading Files -------

    def readfiles(self, file , entropy = False):
        with open(file,'r') as fh:
            lines = fh.readlines()
        return [line.rstrip('\n') for line in lines]

    #----- Green Gray or Yellow
    def isMatched(self,text, pattern):
        if re.search(pattern, text):
            return True
        return False
    
    def gray(self, grayset, allowed = None):
        if allowed is None:
            allowed = self._passed

        if grayset == "":
            return allowed
        
        pattern = re.compile('[{0}]'.format(grayset))
        passed_ = []
        for word in allowed:
            if not self.isMatched(word, pattern):
                passed_.append(word)
        
        return passed_


    def green(self,greenList,allowed):
        passed_ = []       
        if greenList == []:
            return allowed
        else:
            greenList = list(greenList)
            char , position = greenList[0]
            for word in allowed:
                if word[position] == char:
                    passed_.append(word)
            return self.green(greenList[1:],passed_)
    
    def notGreen(self,greenList,allowed):
        passed_ = []       
        if greenList == []:
            return allowed
        else:
            char , position = greenList[0]
            for word in allowed:
                if word[position] != char:
                    passed_.append(word)
            return self.notGreen(greenList[1:],passed_)

    def yellow(self, yellowList,yellowset,allowed):

        if yellowset == "":
            return allowed
        
        pattern = re.compile('[{0}]'.format(yellowset))
        passed_ = []
        for word in allowed:
            if self.isMatched(word, pattern):
                passed_.append(word)
        passed_ = self.notGreen(yellowList,passed_)
        return passed_

    #---- WordleWord Test

    def compareWord(self,guess,word, tup = False):
        gray_ = ""
        yellow_ = ""
        yellowList_ = set()
        greenList_ = set()
        
        for i in range(0,5):
            if guess[i] == word[i]:
                greenList_.add((guess[i],i))
            else:
                if guess[i] in word:
                    if guess[i] not in yellow_:
                        yellow_ += guess[i]
                    yellowList_.add((guess[i],i))
                if guess[i] not in word and guess[i] not in gray_:
                    gray_ += guess[i]
        
        if tup:
            return (gray_, tuple(yellowList_), tuple(greenList_))
        else:
            return {    "gray"      : gray_ ,
                    "yellow"    : yellow_,
                    "yellowList" : yellowList_,
                    "greenList" : greenList_ }

    
    def bestGuess(self,allowed,chance):
        if len(allowed) == 1:
            return allowed.pop()
        if chance > 0:
            guess = random.choices(allowed, weights = [self._allowedWordEntropy[e]  for e in allowed])[0]
            return guess
        else:
            guess = random.choice(allowed)
            return guess
        

    def reduce(self, guess_word,allowed):
        meta = self.compareWord(guess_word,self._wordleWord)
        allowed = self.green(list(meta['greenList']),allowed)
        allowed = self.gray(meta['gray'],allowed)
        allowed = self.yellow(list(meta["yellowList"]),meta['yellow'],allowed)
        #print(len(allowed))
        return allowed

    # ---- Remembering Old
    def calEntropy(self,wordprob):
        entropy = {} 
        for key,value in wordprob.items():
            entropy.update({key : value * math.log2(1/value)})
        return entropy

    def letRemember(self):
        try:
            with open("word_frequency.txt","a") as fh:
                fh.write(self._wordleWord + "\n")

        except AttributeError:
            pass

    def updateEnropy(self):
        word_frequency = self.readfiles("word_frequency.txt")
        total_count = 2315 #len(word_frequency)
        word_frequency = Counter(word_frequency)
        wordprob = {key: value/total_count for key, value in word_frequency.items()}
        entropy = self.calEntropy(wordprob)

        for key, value in entropy.items():
            self._allowedWordEntropy[key] += value
    
    # ----- Run

    def run(self,chance = 6, train = False):
        self._wordleWord = random.choice(self._answers)
        if train:
            self.letRemember()
        print(self._wordleWord.upper())

        allowed = self._passed
        while len(allowed) != 0:
            if chance == 0:
                break
            chance -= 1
            guess_word = self.bestGuess(allowed,chance)
            print(guess_word.upper(), end=' ')
            allowed = self.reduce(guess_word, allowed)
        print()

In [128]:
gm = Wordle()

gm.letRemember()

In [132]:
for i in range(2000):
    gm.run()

DUTCH
ABBES OPIUM GUTTY CUTCH DUTCH DUTCH 
AZURE
DOSED MARGE FRIZE PEARE LUCRE ENURE 
SLIME
CARTE OPEPE NEUME GLIME SLIME 
TROOP
PRATY TROPE TROMP TROOP 
FREED
SEXES HAWED LOMED TYRED DRIED URNED 
REPAY
BUNDH ROARY RITZY REPAY REPAY 
BRAWN
LIMBS BENTO BAJAN BRAWN 
DREAM
STELL EXEME GREEK PREON AREAD DREAM 
NOMAD
SKILL ROWAN NOYAU NOMAD NOMAD 
GOOFY
SERVE CILIA POYNT DOGGY GOOBY GOOKY 
THYME
BESET ODEUM MATTE CRIME THYME 
LEMUR
MOOKS AMATE PILUM FEMUR DEMUR LEMUR 
CRAZY
RALLY GOORY CRAZY CRAZY 
MINCE
FEALS GRYCE PUNCE WINCE MINCE 
GIVEN
FAVAS CUVEE NOVEL ERVEN GIVEN GIVEN 
BLEND
NAVES JEMBE FLECK ELEGY PLEON BLEND 
AMUSE
MOILE GRAME SMEKE AMUSE AMUSE 
GIRTH
MAMEY DROID TWILT GITCH GIRTH 
FEIGN
CURBS PLOAT MENGE FEIGN FEIGN 
BRIAR
SPUNK THYMI DRIVE GRILL FRIZZ BRIAR 
MEALY
EANED BERME PEALS YEALM MEALY MEALY 
BLURB
MANGE DIFFS BURRY BLURT BLURB 
MACHO
DIMLY CHEMO MOTTO MANGO MACHO MACHO 
GRAZE
PORTY LAGER ARUMS WREAK BRAVE CRARE 
MINUS
COFFS WIKIS MILDS MIGGS MIRVS MINUS 
MORAL
GLIAS TAM